In [22]:
#Importing Libraries
import pandas as pd
import random
import pickle
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten,BatchNormalization, Dropout, Lambda, Conv2D, MaxPool2D, InputLayer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import VGG16
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.preprocessing import LabelEncoder
import random 

random.seed(19)

In [23]:
# Load the saved model later
RMS_Model = tf.keras.models.load_model('RMS_Model.keras')

In [24]:
RMS_Model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 64, 64, 128)    │         6,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 32, 32, 64)     │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 16, 16, 32)     │        32,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 8, 8, 16)       │         8,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 430,882 (1.64 MB)

 Trainable params: 215,440 (841.56 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 215,442 (841.57 KB)

### RMSProp Optimizer Model

In [25]:
IMG_SIZE = 64
DIR = 'target'
for img in os.listdir(DIR):
    print(img)
    img_arr = cv2.imread(os.path.join(DIR,img))
    img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
    #plt.imshow(img_arr)
    new_array = cv2.resize(img_arr, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array)
    new_array = new_array/255.0
    new_array = np.array(new_array).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    predicted_sign = RMS_Model.predict(new_array)[0]
    predicted_label = np.argmax(predicted_sign)
    print("Predicted Label: ", predicted_label)

    label_mapping = {
        0: 'END OF EXP WAY', 1: 'NO JAYWALKING', 2: 'TP CAMERA ZONE', 3: 'SPD LIMIT 90',
        4: 'TURN LEFT', 5: 'ERP', 6: 'SPD LIMIT 70', 7: 'U TURN LANE', 8: 'SPLIT WAY',
        9: 'STOP', 10: 'SPD LIMIT 50', 11: 'CURVE RIGHT ALIGNMENT MARKER', 12: 'ZEBRA CROSSING',
        13: 'RAIN SHELTER', 14: 'NO ENTRY', 15: 'KEEP LEFT', 16: 'PARKING AREA FOR MOTORCARS',
        17: 'PEDESTRIAN USE CROSSING', 18: 'RESTRICTED ZONE AHEAD', 19: 'CURVE LEFT ALIGNMENT MARKER',
        20: 'START OF EXP WAY', 21: 'GIVE WAY', 22: 'NO VEH OVER HEIGHT 4.5', 23: 'SPD LIMIT 40',
        24: 'SLOW SPEED', 25: 'ROAD HUMP', 26: 'NO LEFT TURN', 27: 'ONE WAY RIGHT', 28: 'ONE WAY LEFT',
        29: 'SLOW DOWN', 30: 'MERGE', 31: 'NO RIGHT TURN'
    }

    print("Predicted Sign: ",label_mapping[predicted_label])
    print("Predicted Probability: ", round(predicted_sign[predicted_label],2)*100, "%")
    print()

keep_left.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Predicted Label:  15
Predicted Sign:  KEEP LEFT
Predicted Probability:  100.0 %

end_expressway.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Label:  0
Predicted Sign:  END OF EXP WAY
Predicted Probability:  100.0 %

speed_limit_50.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Label:  10
Predicted Sign:  SPD LIMIT 50
Predicted Probability:  98.00000190734863 %

parking_car.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Label:  16
Predicted Sign:  PARKING AREA FOR MOTORCARS
Predicted Probability:  100.0 %

curve_right.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Label:  11
Predicted Sign:  CURVE RIGHT ALIGNMENT MARKER
Predicted Probability:  100.0 %

tp_camera.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Label:  2
Predicted Sign:  TP CAMERA ZONE
Predicted Probability:  100.0 %



### Adam Optimiser Classifier

In [26]:
Adam_Model = tf.keras.models.load_model('Adam_Model.keras')

In [27]:
Adam_Model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 128)    │         6,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 64)     │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 32)     │        32,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 16)       │         8,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 646,322 (2.47 MB)

 Trainable params: 215,440 (841.56 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 430,882 (1.64 MB)

In [28]:
IMG_SIZE = 64
DIR = 'target'
for img in os.listdir(DIR):
    print(img)
    img_arr = cv2.imread(os.path.join(DIR,img))
    img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
    #plt.imshow(img_arr)
    new_array = cv2.resize(img_arr, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array)
    new_array = new_array/255.0
    new_array = np.array(new_array).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    predicted_sign = Adam_Model.predict(new_array)[0]
    predicted_label = np.argmax(predicted_sign)
    print("Predicted Label: ", predicted_label)

    label_mapping = {
        0: 'END OF EXP WAY', 1: 'NO JAYWALKING', 2: 'TP CAMERA ZONE', 3: 'SPD LIMIT 90',
        4: 'TURN LEFT', 5: 'ERP', 6: 'SPD LIMIT 70', 7: 'U TURN LANE', 8: 'SPLIT WAY',
        9: 'STOP', 10: 'SPD LIMIT 50', 11: 'CURVE RIGHT ALIGNMENT MARKER', 12: 'ZEBRA CROSSING',
        13: 'RAIN SHELTER', 14: 'NO ENTRY', 15: 'KEEP LEFT', 16: 'PARKING AREA FOR MOTORCARS',
        17: 'PEDESTRIAN USE CROSSING', 18: 'RESTRICTED ZONE AHEAD', 19: 'CURVE LEFT ALIGNMENT MARKER',
        20: 'START OF EXP WAY', 21: 'GIVE WAY', 22: 'NO VEH OVER HEIGHT 4.5', 23: 'SPD LIMIT 40',
        24: 'SLOW SPEED', 25: 'ROAD HUMP', 26: 'NO LEFT TURN', 27: 'ONE WAY RIGHT', 28: 'ONE WAY LEFT',
        29: 'SLOW DOWN', 30: 'MERGE', 31: 'NO RIGHT TURN'
    }

    print("Predicted Sign: ",label_mapping[predicted_label])
    print("Predicted Probability: ", round(predicted_sign[predicted_label],2)*100, "%")
    print()

keep_left.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted Label:  15
Predicted Sign:  KEEP LEFT
Predicted Probability:  100.0 %

end_expressway.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label:  0
Predicted Sign:  END OF EXP WAY
Predicted Probability:  100.0 %

speed_limit_50.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Label:  10
Predicted Sign:  SPD LIMIT 50
Predicted Probability:  63.999998569488525 %

parking_car.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label:  16
Predicted Sign:  PARKING AREA FOR MOTORCARS
Predicted Probability:  100.0 %

curve_right.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label:  11
Predicted Sign:  CURVE RIGHT ALIGNMENT MARKER
Predicted Probability:  100.0 %

tp_camera.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Label:  2
Predicted Sign:  TP CAMERA ZONE
Predicted Probability:  100.0 %

